In [ ]:
import pandas as pd
import numpy as np
import warnings
import os
from datetime import datetime, timedelta

def time_exit_11_bar(entry_index: int, df):

    """
    Closes a position after exactly 11 bars from entry.

    The function calculates the exit index by adding 11 bars to the entry index
    and automatically handles cases where the exit would exceed DataFrame boundaries.

    Parameters:
    -----------
    entry_index : int
        Index of the bar where the position was opened
    df : pandas.DataFrame
        DataFrame containing market data

    Returns:
    --------
    tuple (exit_index, exit_row)
        exit_index : int
            Index of the exit bar
        exit_row : pandas.Series
            DataFrame row corresponding to the exit bar

    Example:
    --------
    >>> exit_idx, exit_data = time_exit_11_bar(100, df)
    >>> print(f"Exit at bar {exit_idx}")
    Exit at bar 111

    Notes:
    ------
    - If entry_index + 11 exceeds DataFrame length, exits at the last available bar
    - Uses zero-based indexing (common in Python/pandas)
    - The 11-bar count includes the entry bar as bar 0
    """

    exit_index = entry_index + 11

    if exit_index >= len(df):
        exit_index = len(df) - 1

    exit_row = df.iloc[exit_index]
    return exit_index, exit_row


def profitable_close(entry_index: int, df):

    """
    Finds the exit point after 5 consecutive profitable closes following the entry index.

    A profitable close is defined as a close price higher than the previous bar's close.
    The function scans forward from the entry bar to find the first occurrence of
    5 consecutive profitable closes and returns the corresponding exit point.

    Parameters:
    -----------
    entry_index : int
        Index of the bar where the position was opened
    df : pandas.DataFrame
        DataFrame containing market data with 'Close' column

    Returns:
    --------
    pandas.DataFrame or None
        DataFrame slice from entry bar to exit bar (inclusive) if 5 consecutive
        profitable closes are found, otherwise None

    Example:
    --------
    >>> trade_data = profitable_close(150, df)
    >>> if trade_data is not None:
    >>>     print(f"Trade duration: {len(trade_data)} bars")
    >>>     print(f"Exit price: {trade_data['Close'].iloc[-1]}")

    Notes:
    ------
    - The function modifies the input DataFrame by adding a 'profitable' column
    - Scanning starts from the bar immediately after the entry bar (entry_index + 1)
    - Consecutive profitable closes must be exactly 5 in a row without interruptions
    - If no streak is found, returns None and prints a warning message
    - The returned slice includes both entry and exit bars
    """

    df['profitable'] = df['close'] > df['close'].shift(1)

    start_idx = entry_index + 1
    consecutive = 0
    exit_idx = None
    for i in range(start_idx, len(df)):
        if df['profitable'].iloc[i]:
            consecutive += 1
            if consecutive == 5:
                exit_idx = i
                print(f"Exit at the bar {i} after 5 consecutive profitable closes")
                break
        else:
            consecutive = 0
    if exit_idx:
        return df.iloc[entry_index:exit_idx + 1]
    else:
        print("No sequence of 5 consecutive profitable closes found.")
        return None


def Error_handling(path: str, target_columns=None):

    """
    Loads a CSV file and validates the presence and quality of required columns.

    Args:
        path (str): The path to the CSV file to be loaded.
        target_columns (list, optional): List of column names expected in the file. If not provided, uses default columns ['date', 'open', 'high', 'low', 'close', 'volume'].

    Returns:
        pd.DataFrame: The loaded DataFrame after validation.

    Raises:
        ValueError: If the file path is missing, the file cannot be read, or required columns are missing.
        FileNotFoundError: If the specified file does not exist.

    Warns:
        UserWarning: If 'path' or 'target_columns' parameters are missing.
        UserWarning: If any of the required columns contain missing data.

    Behavior:
        - Checks that a file path is provided and the file exists.
        - Reads the CSV file into a DataFrame.
        - Verifies the presence of all required columns; raises an error if any are missing.
        - Issues warnings if required parameters are missing or if any expected columns have missing (NaN) values.

    """

    if not path:
        warnings.warn("Path parameter is missing.")
        raise ValueError("File path must be provided.")

    if not os.path.isfile(path):
        raise FileNotFoundError(f"File not found: {path}")

    if target_columns is None:
        warnings.warn("target_columns parameter is missing. Using default columns.")
        target_columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    try:
        df = pd.read_csv(path)
    except Exception as e:
        raise ValueError(f"Error reading the CSV file: {str(e)}")

    missing = [col for col in target_columns if col not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    for col in target_columns:
        if df[col].isnull().any():
            warnings.warn(f"Missing data in column: {col}")

    return df


def create_mock_data_1():
    """Create first mock dataset with predictable pattern"""
    np.random.seed(42)
    dates = pd.date_range(start='2024-01-01', periods=20, freq='D')

    data = {
        'date': dates,
        'open': np.random.uniform(100, 110, 20),
        'high': np.random.uniform(110, 120, 20),
        'low': np.random.uniform(90, 100, 20),
        'close': np.linspace(100, 120, 20),
        'volume': np.random.randint(1000, 10000, 20)
    }

    return pd.DataFrame(data)

def create_mock_data_2():
    """Create second mock dataset with specific pattern for profitable_close testing"""
    np.random.seed(123)
    dates = pd.date_range(start='2024-01-01', periods=30, freq='D')

    # Create a specific pattern for close prices
    close_prices = [100]
    for i in range(1, 30):
        if i in [5, 6, 7, 8, 9]:  # Create 5 consecutive profitable closes
            close_prices.append(close_prices[-1] + 2)
        elif i in [15, 16, 17, 18, 19]:  # Another sequence
            close_prices.append(close_prices[-1] + 1.5)
        else:
            # Random but mostly increasing
            change = np.random.choice([-0.5, 0.5, 1, 1.5], p=[0.2, 0.3, 0.3, 0.2])
            close_prices.append(close_prices[-1] + change)

    data = {
        'date': dates,
        'open': [p - np.random.uniform(0.5, 2) for p in close_prices],
        'high': [p + np.random.uniform(1, 3) for p in close_prices],
        'low': [p - np.random.uniform(1, 3) for p in close_prices],
        'close': close_prices,
        'volume': np.random.randint(1000, 10000, 30)
    }

    return pd.DataFrame(data)


# Test time_exit_11_bar
print("=== Testing time_exit_11_bar ===")
df1 = create_mock_data_1()
print(f"DataFrame length: {len(df1)}")

# Test case 1: Normal case
print("\nTest 1: Entry at index 5")
entry_idx = 5
exit_idx, exit_row = time_exit_11_bar(entry_idx, df1)
print(f"Entry index: {entry_idx}, Exit index: {exit_idx}")
print(f"Expected exit index: {entry_idx + 11}")
print(f"Exit date: {exit_row['date']}")
print(f"Exit close price: {exit_row['close']:.2f}")

# Test case 2: Edge case
print("\nTest 2: Entry at index 15 (near end)")
entry_idx = 15
exit_idx, exit_row = time_exit_11_bar(entry_idx, df1)
print(f"Entry index: {entry_idx}, Exit index: {exit_idx}")
print(f"Expected exit index: {len(df1) - 1} (last index)")
print(f"Exit date: {exit_row['date']}")
print(f"Exit close price: {exit_row['close']:.2f}")

# Test profitable_close
print("\n=== Testing profitable_close ===")
df2 = create_mock_data_2()
print(f"DataFrame length: {len(df2)}")

# Add profitable column for visualization
df2['profitable'] = df2['close'] > df2['close'].shift(1)
print("\nFirst 15 rows with profitable flag:")
print(df2[['date', 'close', 'profitable']].head(15).to_string())

# Test case 1: Entry where we expect to find 5 consecutive profitable closes
print("\nTest 1: Entry at index 4 (should find sequence at indices 5-9)")
entry_idx = 4
result = profitable_close(entry_idx, df2)

if result is not None:
    print(f"Trade found! Duration: {len(result)} bars")
    print(f"Entry date: {result.iloc[0]['date']}")
    print(f"Exit date: {result.iloc[-1]['date']}")
    print(f"Entry price: {result.iloc[0]['close']:.2f}")
    print(f"Exit price: {result.iloc[-1]['close']:.2f}")
    print(f"Return: {(result.iloc[-1]['close'] - result.iloc[0]['close']):.2f}")
else:
    print("No trade sequence found")

# Test Error_handling
print("\n=== Testing Error_handling ===")

# Create a temporary CSV file for testing
test_csv_path = "test_mock_data.csv"
df1.to_csv(test_csv_path, index=False)

try:
    # Test case 1: Normal loading
    print("Test 1: Normal CSV loading")
    loaded_df = Error_handling(test_csv_path)
    print(f"Successfully loaded DataFrame with shape: {loaded_df.shape}")
    print(f"Columns: {list(loaded_df.columns)}")

    # Test case 2: With custom target columns
    print("\nTest 2: Custom target columns")
    custom_columns = ['date', 'open', 'close']
    loaded_df = Error_handling(test_csv_path, target_columns=custom_columns)
    print(f"Successfully validated custom columns")

    # Test case 3: Missing file
    print("\nTest 3: Missing file (should raise error)")
    try:
        Error_handling("non_existent_file.csv")
    except FileNotFoundError as e:
        print(f"Expected error: {e}")

    # Test case 4: Missing columns
    print("\nTest 4: Missing columns (should raise error)")
    try:
        Error_handling(test_csv_path, target_columns=['date', 'open', 'nonexistent_column'])
    except ValueError as e:
        print(f"Expected error: {e}")

finally:
    # Clean up
    if os.path.exists(test_csv_path):
        os.remove(test_csv_path)
        print(f"\nCleaned up test file: {test_csv_path}")

# ## 6. Integration Test
print("\n=== Integration Test ===")

# Test both exit strategies on the same dataset
df_test = create_mock_data_2()
entry_idx = 8

print(f"Testing both exit strategies from entry index: {entry_idx}")
print(f"Entry date: {df_test.iloc[entry_idx]['date']}")
print(f"Entry price: {df_test.iloc[entry_idx]['close']:.2f}")

# Time-based exit
time_exit_idx, time_exit_row = time_exit_11_bar(entry_idx, df_test)
print(f"\nTime-based exit:")
print(f"  Exit index: {time_exit_idx}")
print(f"  Exit date: {time_exit_row['date']}")
print(f"  Exit price: {time_exit_row['close']:.2f}")
print(f"  Holding period: {time_exit_idx - entry_idx} bars")

# Profit-based exit
profit_trade = profitable_close(entry_idx, df_test)
if profit_trade is not None:
    print(f"\nProfit-based exit:")
    print(f"  Exit index: {profit_trade.index[-1]}")
    print(f"  Exit date: {profit_trade.iloc[-1]['date']}")
    print(f"  Exit price: {profit_trade.iloc[-1]['close']:.2f}")
    print(f"  Holding period: {len(profit_trade) - 1} bars")

    # Compare strategies
    time_return = time_exit_row['close'] - df_test.iloc[entry_idx]['close']
    profit_return = profit_trade.iloc[-1]['close'] - profit_trade.iloc[0]['close']
    print(f"\nStrategy Comparison:")
    print(f"  Time-based return: {time_return:.2f}")
    print(f"  Profit-based return: {profit_return:.2f}")
else:
    print(f"\nProfit-based exit: No sequence found")

# ## 7. Simple Validation Tests

def run_validation_tests():
    print("=== VALIDATION TESTS ===")

    # Test 1: time_exit_11_bar con dati semplici
    simple_df = pd.DataFrame({'close': range(100, 115)})
    exit_idx, exit_data = time_exit_11_bar(0, simple_df)
    assert exit_idx == 11, f"Expected 11, got {exit_idx}"
    assert exit_data['close'] == 111, f"Expected 111, got {exit_data['close']}"
    print(" time_exit_11_bar test passed")

    # Test 2: profitable_close
    profit_df = pd.DataFrame({
        'close': [100, 101, 102, 103, 104, 105, 106, 104, 107, 108]
    })
    result = profitable_close(0, profit_df)
    assert result is not None, "Expected to find profitable sequence"

    expected_length = 6
    actual_length = len(result)
    assert actual_length == expected_length, f"Expected {expected_length} bars, got {actual_length}. Indices: {list(result.index)}"
    print(" profitable_close test passed")

    # Test 3: Error_handling
    import tempfile
    with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as f:
        simple_df.to_csv(f.name, index=False)
        temp_path = f.name

    try:
        loaded = Error_handling(temp_path, ['close'])
        assert len(loaded) == len(simple_df)
        print(" Error_handling test passed")
    finally:
        import os
        os.unlink(temp_path)

    print(" All validation tests passed!")

run_validation_tests()

=== Testing time_exit_11_bar ===
DataFrame length: 20

Test 1: Entry at index 5
Entry index: 5, Exit index: 16
Expected exit index: 16
Exit date: 2024-01-17 00:00:00
Exit close price: 116.84

Test 2: Entry at index 15 (near end)
Entry index: 15, Exit index: 19
Expected exit index: 19 (last index)
Exit date: 2024-01-20 00:00:00
Exit close price: 120.00

=== Testing profitable_close ===
DataFrame length: 30

First 15 rows with profitable flag:
         date  close  profitable
0  2024-01-01  100.0       False
1  2024-01-02  101.0        True
2  2024-01-03  101.5        True
3  2024-01-04  102.0        True
4  2024-01-05  103.0        True
5  2024-01-06  105.0        True
6  2024-01-07  107.0        True
7  2024-01-08  109.0        True
8  2024-01-09  111.0        True
9  2024-01-10  113.0        True
10 2024-01-11  114.0        True
11 2024-01-12  114.5        True
12 2024-01-13  116.0        True
13 2024-01-14  117.0        True
14 2024-01-15  117.5        True

Test 1: Entry at index 4 

/tmp/ipython-input-1658019181.py:147: UserWarning: target_columns parameter is missing. Using default columns.
  warnings.warn("target_columns parameter is missing. Using default columns.")
